# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
import matplotlib
import geopandas as gpd
import jenkspy
import random
import os
from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose

# Define country and parameters

In [ ]:
# Select target country
country = 'Colombia'

# Set country-specific parameters: ISO codes and buffer size (in meters)
if country == 'Argentina':
    country_short = 'ARG'   # ISO 3-letter code
    country_code = 'AR'     # ISO 2-letter code
elif country == 'Chile':
    country_short = 'CHL'
    country_code = 'CL'
elif country == 'Colombia':
    country_short = 'COL'
    country_code = 'CO'
# Uncomment the following if Mexico is to be included in the analysis
# elif country == 'Mexico':
#     country_short = 'MEX'
#     country_code = 'MX'

# Set working directory

In [ ]:
# Define working directory path
wd = (
    '/your/path/to/working/directory/'
)

# Load necessary data
- Functional Urban Area boundaries
- Grid for movement data
- Baseline data

In [ ]:
# Load Functional Urban Areas (FUAs) shapefile and convert to WGS84 coordinate system
gdf_fua = gpd.read_file(
    wd + '/data/inputs/boundaries/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0/'
         'GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg'
).to_crs('EPSG:4326')

# Load national grid shapefile and convert to WGS84 coordinate system
grid = gpd.read_file(
    wd + '/data/inputs/grids/Grid_' + country + '_FB_mov/Grid_' + country + '.shp'
).to_crs('EPSG:4326')

# Toggle for reading raw vs imputed baseline movement data
raw = False

if raw:
    # Load raw baseline movement data
    baseline_mov = pd.read_csv(
        wd + '/data/outputs/' + country_short + '/baseline/baseline_mov.csv'
        # .drop('Unnamed: 0', axis=1)  # Optional cleanup if needed
    )
else:
    # Load imputed baseline movement data with exogenous variables
    baseline_mov_imput = pd.read_csv(
        wd + '/data/outputs/' + country_short +
        '/baseline/movcell-baseline-imput-mov-dist-with-exo-var-flatten.csv'
    ).drop('Unnamed: 0', axis=1)

# Load baseline population data with exogenous variables
baseline_pop_imput = gpd.read_file(
    wd + '/data/outputs/' + country_short +
    '/grids-with-data/movcell-baseline-imput-pop-with-exo-var/'
    'movcell-baseline-imput-pop-with-exo-var.gpkg'
)

# Load movement-distance baseline (non-imputed) with exogenous variables
baseline_mov_dist = pd.read_csv(
    wd + '/data/outputs/' + country_short +
    '/baseline/movcell-baseline-mov-dist-with-exo-var.csv'
).drop(['Unnamed: 0'], axis=1)

# Analysis only for capital? All FUAs? or All FUAs but no capital?

In [ ]:
# Flags to control which Functional Urban Areas (FUAs) to include
capital = False       # If True, analyse only the capital city
no_capital = False    # If True, exclude the capital from the analysis

# Set suffix to be used in file naming or result differentiation
if capital:
    capital_suffix = '_capital'
elif no_capital:
    capital_suffix = '_fuas_no_capital'
else:
    capital_suffix = '_fuas'

# Filter FUAs to only those belonging to the target country
gdf_fua = gdf_fua[gdf_fua['Cntry_ISO'] == country_short].reset_index(drop=True)

# Further filter based on capital/no_capital flags
if capital:
    if country == 'Argentina':
        gdf_fua = gdf_fua[gdf_fua['eFUA_name'] == 'Buenos Aires'].reset_index(drop=True)
    elif country == 'Chile':
        gdf_fua = gdf_fua[gdf_fua['eFUA_name'] == 'Santiago'].reset_index(drop=True)
    elif country == 'Colombia':
        gdf_fua = gdf_fua[gdf_fua['eFUA_name'] == 'Bogota'].reset_index(drop=True)

elif no_capital:
    if country == 'Argentina':
        gdf_fua = gdf_fua[gdf_fua['eFUA_name'] != 'Buenos Aires'].reset_index(drop=True)
    elif country == 'Chile':
        gdf_fua = gdf_fua[gdf_fua['eFUA_name'] != 'Santiago'].reset_index(drop=True)
    elif country == 'Colombia':
        gdf_fua = gdf_fua[gdf_fua['eFUA_name'] != 'Bogota'].reset_index(drop=True)

# Initialise a list to store index of the grid cell with max population density for each FUA
index_fuas = []

# Loop over each FUA to find the grid cell with the highest population density
for i in range(len(gdf_fua)):
    gdf_fua_join = gdf_fua.copy()
    grid_join = grid.copy()

    # Spatial join: match grid cells intersecting with the current FUA
    grid_fua = gdf_fua_join.iloc[[i]].sjoin(grid_join, how="left", predicate='intersects')

    # Extract FID indexes of intersecting grid cells
    indexes_fua = np.array(grid_fua['FID'])

    # Subset the population baseline using those indexes
    baseline_pop_fua = baseline_pop_imput.iloc[indexes_fua]

    # Find the index of the grid cell with the maximum population density
    max_density = max(baseline_pop_fua['density'])
    index_fua = baseline_pop_fua[baseline_pop_fua['density'] == max_density].index[0]

    # Store this index for later reference
    index_fuas.append(index_fua)

# Add the list of center grid cell indexes as a new column in gdf_fua
gdf_fua['centre'] = index_fuas

In [ ]:
def compute_flows(df_mov_evo, flow_type):
    """
    Compute either outflows or inflows aggregated by origin or destination.

    Parameters:
    - df_mov_evo: DataFrame containing movement data with columns 'O', 'D', and time series data.
    - flow_type: str, either 'outflows' (sum by origin) or 'inflows' (sum by destination).

    Returns:
    - df_flows: DataFrame aggregated by 'O' or 'D'.
    """

    # Initialise df_flows with unique IDs depending on flow_type
    if flow_type == 'outflows':
        df_flows = pd.DataFrame({'O': np.unique(df_mov_evo['O'])})
    elif flow_type == 'inflows':
        df_flows = pd.DataFrame({'D': np.unique(df_mov_evo['D'])})
    else:
        raise ValueError("flow_type must be 'outflows' or 'inflows'")

    # Prepare empty columns for the time series data initialised with NaNs
    time_columns = df_mov_evo.columns[2:]
    df_flows_add = pd.DataFrame({col: [np.nan] * len(df_flows) for col in time_columns})

    # Combine ID column with empty data columns
    df_flows = pd.concat([df_flows, df_flows_add], axis=1)

    # Iterate over each unique ID to compute aggregated flows
    for i in range(len(df_flows)):
        if flow_type == 'outflows':
            ID = df_flows.loc[i, 'O']
            df_subset = df_mov_evo[df_mov_evo['O'] == ID]
        else:  # inflows
            ID = df_flows.loc[i, 'D']
            df_subset = df_mov_evo[df_mov_evo['D'] == ID]

        # Sum values per column, ignoring NaNs
        for column in time_columns:
            values = df_subset[column].dropna()
            if values.empty:
                df_flows.loc[i, column] = np.nan
            else:
                df_flows.loc[i, column] = values.sum()

    return df_flows

In [ ]:
def compute_df_ts(df_flows, df_flows_baseline, initial_col):
    """
    Compute time series of movement sums and baseline sums from flow DataFrames,
    handle missing and zero values by interpolation using nearest observations,
    and calculate rolling averages and percentage changes.

    Parameters:
    - df_flows: DataFrame with flow data over time columns starting at initial_col
    - df_flows_baseline: Baseline DataFrame with same structure as df_flows
    - initial_col: int, index of the first time column in the DataFrames

    Returns:
    - df_ts: DataFrame with dates, movements, baseline, filled values, rolling means,
             and percentage change metrics.
    """

    evo_movs = []
    evo_movs_baseline = []

    # Aggregate sum of movements and baseline for each time column, ignoring NaNs
    for column in df_flows.columns[initial_col:]:
        sums_mov = []
        sums_baseline = []
        for i in range(len(df_flows)):
            val_mov = df_flows.loc[i, column]
            val_base = df_flows_baseline.loc[i, column]
            if not pd.isna(val_mov) and not pd.isna(val_base):
                sums_mov.append(val_mov)
                sums_baseline.append(val_base)
        if sums_mov:
            evo_movs.append(np.sum(sums_mov))
            evo_movs_baseline.append(np.sum(sums_baseline))
        else:
            evo_movs.append(np.nan)
            evo_movs_baseline.append(np.nan)

    # Create DataFrame with date and aggregated sums
    df_ts = pd.DataFrame({
        'date': df_flows.columns[initial_col:],
        'movements': evo_movs,
        'baseline': evo_movs_baseline
    })

    # Function to fill zeros and NaNs using mean of closest 15 observations in time series
    def fill_zeros_and_nans(series, fill_column_name):
        series.replace(0, np.nan, inplace=True)
        series.replace([np.inf, -np.inf], np.nan, inplace=True)

        is_na = series.isna()
        filled_series = series.copy()

        # Extract rows where values are not NaN for reference
        valid_idx = series.dropna().index
        valid_vals = series.dropna()

        # For each NaN, find 15 nearest valid observations by index distance and take their mean
        for idx in series[is_na].index:
            distances = abs(valid_idx - idx)
            nearest_idx = distances.nsmallest(15).index
            filled_series.loc[idx] = valid_vals.loc[nearest_idx].mean()

        return filled_series

    # Fill movements and baseline columns
    df_ts['movements_fill'] = fill_zeros_and_nans(df_ts['movements'], 'movements_fill')
    df_ts['rolling'] = df_ts['movements_fill'].rolling(window=15, min_periods=1).mean()

    df_ts['baseline_fill'] = fill_zeros_and_nans(df_ts['baseline'], 'baseline_fill')
    df_ts['rolling_baseline'] = df_ts['baseline_fill'].rolling(window=15, min_periods=1).mean()

    # Calculate percentage change between movements and baseline
    df_ts['perchange'] = (df_ts['movements_fill'] - df_ts['baseline_fill']) / df_ts['baseline_fill'] * 100
    df_ts['rolling_perchange'] = df_ts['perchange'].rolling(window=30, min_periods=1).mean()

    return df_ts

In [ ]:
def compute_df_ts_weekly(df_ts):
    """
    Aggregate daily time series data into weekly summaries.

    Parameters:
    - df_ts: DataFrame containing daily time series with at least the following columns:
             'date', 'movements', 'baseline', 'movements_fill', 'baseline_fill'

    Returns:
    - df_ts_weekly: DataFrame aggregated by week with sums and percentage change.
    """

    # Calculate number of full weeks in the data (each week has 7 days)
    num_weeks = len(df_ts) // 7

    # Initialise DataFrame for weekly aggregation
    df_ts_weekly = pd.DataFrame({'week_no': range(num_weeks)})

    # Aggregate by week:
    # - 'week_start' is the date of the first day of the week
    # - sum the columns over each week block of 7 days
    df_ts_weekly['week_start'] = [df_ts.loc[i * 7, 'date'] for i in range(num_weeks)]
    df_ts_weekly['movements'] = [np.sum(df_ts.loc[i * 7:(i + 1) * 7 - 1, 'movements']) for i in range(num_weeks)]
    df_ts_weekly['baseline'] = [np.sum(df_ts.loc[i * 7:(i + 1) * 7 - 1, 'baseline']) for i in range(num_weeks)]
    df_ts_weekly['movements_fill'] = [np.sum(df_ts.loc[i * 7:(i + 1) * 7 - 1, 'movements_fill']) for i in range(num_weeks)]
    df_ts_weekly['baseline_fill'] = [np.sum(df_ts.loc[i * 7:(i + 1) * 7 - 1, 'baseline_fill']) for i in range(num_weeks)]

    # Compute weekly percentage change between filled movements and baseline
    df_ts_weekly['perchange'] = [
        (df_ts_weekly.loc[i, 'movements_fill'] - df_ts_weekly.loc[i, 'baseline_fill']) / df_ts_weekly.loc[i, 'baseline_fill'] * 100
        for i in range(num_weeks)
    ]

    return df_ts_weekly

# Read some additional data

In [ ]:
# Load COVID-19 stringency data from CSV file
df_stringency = pd.read_csv(wd + '/data/inputs/covid-stringency/owid-covid-data.csv')

# Filter the data for the specified country (case-insensitive, capitalizes first letter)
df_stringency = df_stringency[df_stringency['location'] == str(country).capitalize()].reset_index(drop=True)                             

# Set filename suffixes based on processing options:
- `dist`: whether to include movements with distance >=0 (adds '_dist' if True) <br>
- `raw`: whether using raw data (adds '_raw' if True) <br>
- `adjust`: whether data is adjusted (adds '_adjust' if True) <br>

In [ ]:
# Flags to control data type and processing options
dist = True     # If True, use distance-based data
raw = False     # If True, use raw (non-imputed) data
adjust = True   # If True, use adjusted data (e.g., adjusted for exogenous variables)

# Set suffix based on distance flag
if dist is True:
    dist_suffix = '_dist'
else:
    dist_suffix = ''

# Set suffix based on raw flag
if raw is True:
    raw_suffix = '_raw'
else:
    raw_suffix = ''
    
    # Only consider adjustment suffix if raw is False (i.e., imputed data is used)
    if adjust is True:
        adjust_suffix = '_adjust'
    else:
        adjust_suffix = ''

# Read movement data as time series for each tile

In [ ]:
# Load mobility evolution data based on configured suffixes
# File includes options for distance-based, raw, or adjusted versions
df_mov_evo = pd.read_csv(
    wd + '/data/outputs/' + country_short + '/evo/mov_evo' + dist_suffix + raw_suffix + adjust_suffix + '.csv'
).drop('Unnamed: 0', axis=1)

# Load corresponding baseline mobility evolution data
# Note: No adjusted version for baseline file
df_mov_evo_baseline = pd.read_csv(
    wd + '/data/outputs/' + country_short + '/evo/mov_evo_baseline' + dist_suffix + raw_suffix + '.csv'
).drop('Unnamed: 0', axis=1)


# Read movement data as time series for inflows or outflows

In [ ]:
 # Set the type of flow to analyze ('movs', 'inflows', or 'outflows')
flows = 'outflows'  # change to 'movs', 'inflows', or 'outflows' as needed

# Load appropriate flow data depending on the selected flow type
if flows in ['inflows', 'outflows']:
    # Load computed flow data and baseline from CSV, dropping the index column
    df_flows = pd.read_csv(
        wd + '/data/outputs/' + country_short + '/mov-analysis/' +
        flows + dist + raw + adjust + '.csv'
    ).drop('Unnamed: 0', axis=1)

    df_flows_baseline = pd.read_csv(
        wd + '/data/outputs/' + country_short + '/mov-analysis/' +
        flows + '_baseline' + dist + raw + '.csv'
    ).drop('Unnamed: 0', axis=1)

    # Set the initial column index for further processing
    initial_col = 1

else:
    # Use raw mobility evolution data directly if 'movs' is selected
    df_flows = df_mov_evo
    df_flows_baseline = df_mov_evo_baseline
    initial_col = 2


# Set distance within a radius of 100 of FUAs centres? True for YES, False for NO

In [ ]:
# Enable radius filtering
radius = True

# Define suffix based on radius flag
radius_suffix = '_radius' if radius else ''

# Mask flows to include only those connected to FUA centres (either origin or destination)
mask = df_flows['O'].isin(gdf_fua['centre']) | df_flows['D'].isin(gdf_fua['centre'])
df_flows_fua = df_flows[mask].reset_index(drop=True)
df_flows_fua_baseline = df_flows_baseline[mask].reset_index(drop=True)

# Prepare to drop flows that exceed a specified distance (100 km)
index_to_drop = []

if radius:
    for i in range(len(df_flows_fua)):
        O = df_flows_fua.loc[i, 'O']
        D = df_flows_fua.loc[i, 'D']
        
        # Get distance for weekday 0 (e.g., Monday) between origin and destination
        dist = baseline_mov_dist[
            (baseline_mov_dist['O'] == O) &
            (baseline_mov_dist['D'] == D) &
            (baseline_mov_dist['wday'] == 0)
        ].reset_index(drop=True).loc[0, 'dist']
        
        # Drop if distance is greater than or equal to 100 km
        if dist >= 100000:
            index_to_drop.append(i)

# Apply distance filter
df_flows_fua = df_flows_fua.drop(index_to_drop).reset_index(drop=True)
df_flows_fua_baseline = df_flows_fua_baseline.drop(index_to_drop).reset_index(drop=True)

# Starting index for movement data columns (skip ID columns)
initial_col = 4

# Add new columns: 
# 'classify' identifies the grid cell to be classified (not the FUA center)
# 'anchor' identifies the direction (O or D) that contains the FUA center

classify = []
anchor = []

for i in range(len(df_flows_fua)):
    origin = df_flows_fua.loc[i, 'O']
    destination = df_flows_fua.loc[i, 'D']
    
    if origin in list(gdf_fua['centre']):
        if destination in list(gdf_fua['centre']):
            # Both O and D are centres → randomly select one to classify
            random_choice = random.choice([0, 1])
            classify.append(df_flows_fua.loc[i, df_flows_fua.columns[random_choice]])
            anchor.append(df_flows_fua.columns[random_choice])
        else:
            # Origin is centre, classify destination
            classify.append(destination)
            anchor.append('O')
    else:
        # Destination is centre, classify origin
        classify.append(origin)
        anchor.append('D')

# Insert new columns into the dataframes
df_flows_fua.insert(2, 'classify', classify)
df_flows_fua.insert(3, 'anchor', anchor)
df_flows_fua_baseline.insert(2, 'classify', classify)
df_flows_fua_baseline.insert(3, 'anchor', anchor)

# By density class

In [ ]:
# Number of density classes
n_class_density = 5

# Compute class breaks using Jenks natural breaks optimization
breaks_density = jenkspy.jenks_breaks(
    baseline_pop_imput.dropna(subset=['density'])['density'], 
    n_classes=n_class_density
)

# Slightly adjust first break to ensure inclusion of minimum density value
breaks_density[0] -= 1e-10

# Bin the 'density' column into discrete classes using the computed breaks
baseline_pop_imput['class_density'] = pd.cut(
    baseline_pop_imput['density'], 
    bins=breaks_density, 
    labels=[i for i in range(n_class_density)]
)

# Ensure class labels are numeric
baseline_pop_imput['class_density'] = pd.to_numeric(baseline_pop_imput['class_density'])

# Get unique class labels and number of valid (non-NaN) classes
class_density = np.unique(baseline_pop_imput['class_density'])
n_class_density = len(class_density[~np.isnan(class_density)])

# Initialize matrices to store time series data for each class
n_weeks = int((len(df_flows_fua.columns) - 4) / 7)  # Weekly bins inferred from column structure

df_ts_weekly_inflows_class_density = np.zeros((n_class_density, n_weeks))
df_ts_weekly_inflows_baseline_class_density = np.zeros((n_class_density, n_weeks))
df_ts_weekly_outflows_class_density = np.zeros((n_class_density, n_weeks))
df_ts_weekly_outflows_baseline_class_density = np.zeros((n_class_density, n_weeks))

# Loop through each density class
for i in range(n_class_density):
    # Get indexes of grid cells in this density class
    indexes = set(baseline_pop_imput[baseline_pop_imput['class_density'] == i].index)
    
    # -------------------------
    # INFLOWS
    # -------------------------
    # Mask rows where 'classify' cell is in the current class and it's a destination (D)
    mask_inflows = (df_flows_fua['classify'].isin(indexes)) & (df_flows_fua['anchor'] == 'D')
    
    # Filter flows and corresponding baselines
    df_inflows_class_density = df_flows_fua[mask_inflows].reset_index(drop=True)
    df_inflows_class_density_baseline = df_flows_fua_baseline[mask_inflows].reset_index(drop=True)
    
    # Compute time series and weekly aggregates
    df_ts_class_density = compute_df_ts(df_inflows_class_density, df_inflows_class_density_baseline, initial_col)
    df_ts_weekly_inflows = compute_df_ts_weekly(df_ts_class_density)
    
    # Store results
    df_ts_weekly_inflows_class_density[i, :] = df_ts_weekly_inflows['movements']
    df_ts_weekly_inflows_baseline_class_density[i, :] = df_ts_weekly_inflows['baseline']
    
    # -------------------------
    # OUTFLOWS
    # -------------------------
    # Mask rows where 'classify' cell is in the current class and it's an origin (O)
    mask_outflows = (df_flows_fua['classify'].isin(indexes)) & (df_flows_fua['anchor'] == 'O')
    
    # Filter flows and corresponding baselines
    df_outflows_class_density = df_flows_fua[mask_outflows].reset_index(drop=True)
    df_outflows_class_density_baseline = df_flows_fua_baseline[mask_outflows].reset_index(drop=True)
    
    # Compute time series and weekly aggregates
    df_ts_class_density = compute_df_ts(df_outflows_class_density, df_outflows_class_density_baseline, initial_col)
    df_ts_weekly_outflows = compute_df_ts_weekly(df_ts_class_density)
    
    # Store results
    df_ts_weekly_outflows_class_density[i, :] = df_ts_weekly_outflows['movements']
    df_ts_weekly_outflows_baseline_class_density[i, :] = df_ts_weekly_outflows['baseline']

In [ ]:
fig, ax = plt.subplots()

ax.tick_params(
    axis='both', which='both', width=0, length=0, color='k',
    labelsize=20, pad=9
)

viridis = plt.cm.get_cmap('viridis')
norm = plt.Normalize(0, n_class_density - 1)

maxima = []
minima = []

for i in range(n_class_density):

    color = viridis(norm(i))
    inflows_class_density = df_ts_weekly_inflows_class_density[i, :]
    inflows_baseline_class_density = df_ts_weekly_inflows_baseline_class_density[i, :]
    inflows_perchange = [
        (inflows_class_density[i] - inflows_baseline_class_density[i]) /
        inflows_baseline_class_density[i] * 100
        for i in range(len(inflows_baseline_class_density))
    ]
    df_ts_weekly_class_density_plot = pd.DataFrame(
        {'perchange_class': inflows_perchange}
    )

#     df_ts_weekly_class_density_plot.loc[:,'rolling_perchange'] = df_ts_weekly_class_density_plot['perchange_class'].rolling(window=6).mean()
#     ax.plot(np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']))*7, df_ts_weekly_class_density_plot['rolling_perchange'], color=color, lw=2, zorder=3)

    df_ts_weekly_class_density_plot['rolling_perchange'] = (
        df_ts_weekly_class_density_plot['perchange_class']
        .ewm(span=10).mean()
    )
    df_ts_weekly_class_density_plot['rolling_perchange'] = scipy.signal.savgol_filter(
        df_ts_weekly_class_density_plot['rolling_perchange'],
        window_length=10,
        polyorder=3
    )
    ax.plot(
        np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange'])) * 7,
        df_ts_weekly_class_density_plot['rolling_perchange'],
        color=color, lw=5, zorder=3
    )

    maxima.append(max(df_ts_weekly_class_density_plot['rolling_perchange']))
    minima.append(min(df_ts_weekly_class_density_plot['rolling_perchange']))

ax.plot(
    np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    np.zeros(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    linestyle=':', color='k'
)

stringencies = []
for date in df_ts_class_density['date']:
    stringencies.append(
        df_stringency[df_stringency['date'] == date]
        .reset_index(drop=True).loc[0, 'stringency_index']
    )

ymin = -105
ymax = 30

for k in range(len(df_ts_class_density)):
    try:
        rgba = matplotlib.cm.gist_heat(
            1 - (stringencies[k] - min(stringencies)) / max(stringencies)
        )
    except:
        rgba = matplotlib.cm.gist_heat(
            1 - (stringencies[k - 1] - min(stringencies)) / max(stringencies)
        )
    x = [k - 0.50, k + 0.50]
    ax.fill_between(
        x, ymin, ymax, color=rgba, alpha=0.6,
        edgecolor='None', linewidth=0, zorder=0
    )

xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'Apr 2022']
for i in range(0, len(df_ts_class_density['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=23, pad=6, rotation=90)

yticks = []
for i in range(ymin, ymax):
    if i % 25 == 0:
        yticks.append(i)
ax.set_yticks(yticks, yticks)
for y in yticks:
    ax.plot(
        [0, len(df_ts_class_density['rolling_perchange'])],
        [y, y], color='gray', lw=0.7, zorder=0
    )
ax.tick_params(axis='y', labelsize=23, pad=6, rotation=0)

# plt.savefig(wd + '/plots/evolution/' + flows + '/by-density/' + country_short + '/evo_inflows' + capital_suffix + dist_suffix + raw_suffix + adjust_suffix + radius_suffix + '.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.tick_params(
    axis='both', which='both', width=0, length=0,
    color='k', labelsize=20, pad=9
)

viridis = plt.cm.get_cmap('viridis')
norm = plt.Normalize(0, n_class_density - 1)

maxima = []
minima = []

for i in range(n_class_density):

    color = viridis(norm(i))
    outflows_class_density = df_ts_weekly_outflows_class_density[i, :]
    outflows_baseline_class_density = df_ts_weekly_outflows_baseline_class_density[i, :]
    outflows_perchange = [
        (outflows_class_density[i] - outflows_baseline_class_density[i]) /
        outflows_baseline_class_density[i] * 100
        for i in range(len(outflows_baseline_class_density))
    ]
    df_ts_weekly_class_density_plot = pd.DataFrame(
        {'perchange_class': outflows_perchange}
    )

#     df_ts_weekly_class_density_plot.loc[:,'rolling_perchange'] = df_ts_weekly_class_density_plot['perchange_class'].rolling(window=6).mean()
#     ax.plot(np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']))*7, df_ts_weekly_class_density_plot['rolling_perchange'], color=color, lw=2, zorder=3)

    df_ts_weekly_class_density_plot['rolling_perchange'] = (
        df_ts_weekly_class_density_plot['perchange_class']
        .ewm(span=10).mean()
    )
    df_ts_weekly_class_density_plot['rolling_perchange'] = scipy.signal.savgol_filter(
        df_ts_weekly_class_density_plot['rolling_perchange'],
        window_length=10, polyorder=3
    )
    ax.plot(
        np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange'])) * 7,
        df_ts_weekly_class_density_plot['rolling_perchange'],
        color=color, lw=5, zorder=3
    )

    maxima.append(max(df_ts_weekly_class_density_plot['rolling_perchange']))
    minima.append(min(df_ts_weekly_class_density_plot['rolling_perchange']))

ax.plot(
    np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    np.zeros(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    linestyle=':', color='k'
)

stringencies = []
for date in df_ts_class_density['date']:
    stringencies.append(
        df_stringency[df_stringency['date'] == date]
        .reset_index(drop=True).loc[0, 'stringency_index']
    )

ymin = -105
ymax = 30

for k in range(len(df_ts_class_density)):
    try:
        rgba = matplotlib.cm.gist_heat(
            1 - (stringencies[k] - min(stringencies)) / max(stringencies)
        )
    except:
        rgba = matplotlib.cm.gist_heat(
            1 - (stringencies[k - 1] - min(stringencies)) / max(stringencies)
        )
    x = [k - 0.50, k + 0.50]
    ax.fill_between(
        x, ymin, ymax, color=rgba, alpha=0.6,
        edgecolor='None', linewidth=0, zorder=0
    )

xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'Apr 2022']
for i in range(0, len(df_ts_class_density['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=23, pad=6, rotation=90)

yticks = []
for i in range(ymin, ymax):
    if i % 25 == 0:
        yticks.append(i)
ax.set_yticks(yticks, yticks)
for y in yticks:
    ax.plot(
        [0, len(df_ts_class_density['rolling_perchange'])],
        [y, y], color='gray', lw=0.7, zorder=0
    )
ax.tick_params(axis='y', labelsize=23, pad=6, rotation=0)

# plt.savefig(wd + '/plots/evolution/' + flows + '/by-density/' + country_short + '/evo_outflows' + capital_suffix + dist_suffix + raw_suffix + adjust_suffix + radius_suffix + '.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.tick_params(
    axis='both', which='both', width=0, length=0,
    color='k', labelsize=20, pad=9
)

viridis = plt.cm.get_cmap('viridis')
norm = plt.Normalize(0, n_class_density - 1)

maxima = []
minima = []

for i in range(n_class_density):

    color = viridis(norm(i))
    netflows_class_density = (
        df_ts_weekly_inflows_class_density[i, :] -
        df_ts_weekly_outflows_class_density[i, :]
    )
    netflows_baseline_class_density = (
        df_ts_weekly_inflows_baseline_class_density[i, :] -
        df_ts_weekly_outflows_baseline_class_density[i, :]
    )
    netflows_perchange = [
        (netflows_class_density[i] - netflows_baseline_class_density[i]) /
        netflows_baseline_class_density[i] * 100
        for i in range(len(netflows_baseline_class_density))
    ]
    df_ts_weekly_class_density_plot = pd.DataFrame(
        {'perchange_class': netflows_perchange}
    )

#     df_ts_weekly_class_density_plot.loc[:,'rolling_perchange'] = df_ts_weekly_class_density_plot['perchange_class'].rolling(window=6).mean()
#     ax.plot(np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']))*7, df_ts_weekly_class_density_plot['rolling_perchange'], color=color, lw=2, zorder=3)

    df_ts_weekly_class_density_plot['rolling_perchange'] = (
        df_ts_weekly_class_density_plot['perchange_class']
        .ewm(span=10).mean()
    )
    df_ts_weekly_class_density_plot['rolling_perchange'] = scipy.signal.savgol_filter(
        df_ts_weekly_class_density_plot['rolling_perchange'],
        window_length=10, polyorder=3
    )
    ax.plot(
        np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange'])) * 7,
        df_ts_weekly_class_density_plot['rolling_perchange'],
        color=color, lw=5, zorder=3
    )

    maxima.append(max(df_ts_weekly_class_density_plot['rolling_perchange']))
    minima.append(min(df_ts_weekly_class_density_plot['rolling_perchange']))

ax.plot(
    np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    np.zeros(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    linestyle=':', color='k'
)

stringencies = []
for date in df_ts_class_density['date']:
    stringencies.append(
        df_stringency[df_stringency['date'] == date]
        .reset_index(drop=True).loc[0, 'stringency_index']
    )

ymin = -105
ymax = 30

for k in range(len(df_ts_class_density)):
    try:
        rgba = matplotlib.cm.gray(
            1 - (stringencies[k] - min(stringencies)) / max(stringencies)
        )
    except:
        rgba = matplotlib.cm.gray(
            1 - (stringencies[k - 1] - min(stringencies)) / max(stringencies)
        )
    x = [k - 0.50, k + 0.50]
    ax.fill_between(
        x, ymin, ymax, color=rgba, alpha=0.4,
        edgecolor='None', linewidth=0, zorder=0
    )

xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'Apr 2022']
for i in range(0, len(df_ts_class_density['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=23, pad=6, rotation=90)

yticks = []
for i in range(ymin, ymax):
    if i % 25 == 0:
        yticks.append(i)
ax.set_yticks(yticks, yticks)
for y in yticks:
    ax.plot(
        [0, len(df_ts_class_density['rolling_perchange'])],
        [y, y], color='gray', lw=0.7, zorder=0
    )
ax.tick_params(axis='y', labelsize=23, pad=6, rotation=0)

# plt.savefig(wd + '/plots/evolution/' + flows + '/by-density/' + country_short + '/evo_netflows' + capital_suffix + dist_suffix + raw_suffix + adjust_suffix + radius_suffix + '.pdf', bbox_inches = 'tight')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.tick_params(axis='both', which='both', width=0, length=0, color='k', labelsize=20, pad=9)

viridis = plt.cm.get_cmap('viridis')
norm = plt.Normalize(0, n_class_density - 1)

maxima = []
minima = []

for i in range(n_class_density):
    color = viridis(norm(i))

    netflows_class_density = (
        df_ts_weekly_inflows_class_density[i, :] -
        df_ts_weekly_outflows_class_density[i, :]
    )
    netflows_baseline_class_density = (
        df_ts_weekly_inflows_baseline_class_density[i, :] -
        df_ts_weekly_outflows_baseline_class_density[i, :]
    )

    netflows_perchange = [
        (netflows_class_density[j] - netflows_baseline_class_density[j]) /
        netflows_baseline_class_density[j] * 100
        for j in range(len(netflows_baseline_class_density))
    ]

    df_ts_weekly_class_density_plot = pd.DataFrame({
        'perchange_class': netflows_perchange
    })

    # Smoothing with EWM + Savitzky-Golay filter
    df_ts_weekly_class_density_plot['rolling_perchange'] = (
        df_ts_weekly_class_density_plot['perchange_class']
        .ewm(span=10)
        .mean()
    )
    df_ts_weekly_class_density_plot['rolling_perchange'] = scipy.signal.savgol_filter(
        df_ts_weekly_class_density_plot['rolling_perchange'],
        window_length=10,
        polyorder=3
    )

    ax.plot(
        np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange'])) * 7,
        df_ts_weekly_class_density_plot['rolling_perchange'],
        color=color,
        lw=5,
        zorder=3
    )

    maxima.append(max(df_ts_weekly_class_density_plot['rolling_perchange']))
    minima.append(min(df_ts_weekly_class_density_plot['rolling_perchange']))

# Axis limits
xmin = -len(df_ts_class_density) * 0.4
xmax = len(df_ts_class_density) * 1.05
ax.set_xlim([xmin, xmax])

# Horizontal zero line
ax.plot(
    np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    np.zeros(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    linestyle=':',
    color='k'
)

# Background shading from stringency index
stringencies = []
for date in df_ts_class_density['date']:
    stringencies.append(
        df_stringency[df_stringency['date'] == date]
        .reset_index(drop=True)
        .loc[0, 'stringency_index']
    )

ymin = -105
ymax = 30

for k in range(len(df_ts_class_density)):
    try:
        rgba = matplotlib.cm.gray(
            1 - (stringencies[k] - min(stringencies)) / max(stringencies)
        )
    except:
        rgba = matplotlib.cm.gray(
            1 - (stringencies[k - 1] - min(stringencies)) / max(stringencies)
        )
    x = [k - 0.5, k + 0.5]
    ax.fill_between(x, ymin, ymax, color=rgba, alpha=0.4, edgecolor='None', linewidth=0, zorder=0)

# Custom x-ticks
xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'Apr 2022']
for i in range(0, len(df_ts_class_density['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i + 40)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=17, pad=6, rotation=90)

# Custom y-ticks
yticks = []
for i in range(ymin, ymax):
    if i % 25 == 0:
        yticks.append(i)
ax.set_yticks(yticks, yticks)
for y in yticks:
    ax.plot(
        [0, len(df_ts_class_density['rolling_perchange'])],
        [y, y],
        color='gray',
        lw=0.7,
        zorder=0
    )
ax.tick_params(axis='y', labelsize=17, pad=4, rotation=0)

# Secondary y-axis with baseline averages
ax1 = ax.twinx()
ax1.yaxis.tick_left()
netflows_averages = []

for i in range(n_class_density):
    color = viridis(norm(i))
    netflows_baseline_class_density_average = np.mean(
        df_ts_weekly_inflows_baseline_class_density[i, :] -
        df_ts_weekly_outflows_baseline_class_density[i, :]
    )
    ax1.plot(
        [xmin + (0 - xmin) * 0.1, 0 - (0 - xmin) * 0.1],
        [netflows_baseline_class_density_average] * 2,
        color=color,
        lw=5,
        zorder=3
    )
    netflows_averages.append(netflows_baseline_class_density_average)

ymin_ax1 = int(min(netflows_averages) - (max(netflows_averages) - min(netflows_averages)) * 0.2)
ymax_ax1 = int(max(netflows_averages) + (max(netflows_averages) - min(netflows_averages)) * 0.2)
ax1.set_ylim([ymin_ax1, ymax_ax1])

# Custom y-ticks for ax1
yticks_ax1 = []
for i in range(ymin_ax1, ymax_ax1):
    if i % 30000 == 0:
        yticks_ax1.append(i)
ax1.set_yticks(yticks_ax1, [int(y / 1000) for y in yticks_ax1])
ax1.tick_params(axis='both', which='both', width=0, length=0, color='k', pad=9)
ax1.tick_params(axis='y', labelsize=17, pad=6, rotation=0)

# Label for baseline axis
ax1.text(
    xmin + (0 - xmin) * 0.45,
    ymin_ax1 - (ymax_ax1 - ymin_ax1) * 0.15,
    'Pre-pandemic\nbaseline',
    ha='center',
    fontsize=14
)

# Vertical reference line at time zero
ax.yaxis.tick_right()
ax.plot(
    [0.01, 0.01],
    [ymin - (ymax - ymin) * 0.05, ymax + (ymax - ymin) * 0.05],
    linestyle='-',
    color='k',
    lw=2,
    zorder=20
)

# Uncomment to save:
# plt.savefig(
#     wd + '/plots/evolution/' + flows + '/by-density/' + country_short +
#     '/evo_netflows' + capital_suffix + dist_suffix + raw_suffix + adjust_suffix +
#     radius_suffix + '_including_baseline.pdf',
#     bbox_inches='tight'
# )

plt.show()

In [ ]:
netflows_averages

In [ ]:
fig, ax = plt.subplots()

ax.tick_params(axis='both', which='both', width=0, length=0, color='k', labelsize=20, pad=9)

viridis = plt.cm.get_cmap('viridis')
norm = plt.Normalize(0, n_class_density - 1)

maxima = []
minima = []

for i in range(n_class_density):
    color = viridis(norm(i))

    netflows_class_density = (
        df_ts_weekly_inflows_class_density[i, :] -
        df_ts_weekly_outflows_class_density[i, :]
    )
    netflows_baseline_class_density = (
        df_ts_weekly_inflows_baseline_class_density[i, :] -
        df_ts_weekly_outflows_baseline_class_density[i, :]
    )

    netflows_perchange = [
        (netflows_class_density[j] - netflows_baseline_class_density[j]) /
        netflows_baseline_class_density[j] * 100
        for j in range(len(netflows_baseline_class_density))
    ]

    df_ts_weekly_class_density_plot = pd.DataFrame({
        'perchange_class': netflows_perchange
    })

    df_ts_weekly_class_density_plot['rolling_perchange'] = (
        df_ts_weekly_class_density_plot['perchange_class']
        .ewm(span=10)
        .mean()
    )
    df_ts_weekly_class_density_plot['rolling_perchange'] = scipy.signal.savgol_filter(
        df_ts_weekly_class_density_plot['rolling_perchange'],
        window_length=10,
        polyorder=3
    )

    if netflows_averages[i] > 0:
        ax.plot(
            np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange'])) * 7,
            df_ts_weekly_class_density_plot['rolling_perchange'],
            color=color,
            lw=5,
            linestyle=':',
            dashes=[1, 0.5],
            zorder=3
        )
    elif netflows_averages[i] < 0:
        ax.plot(
            np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange'])) * 7,
            df_ts_weekly_class_density_plot['rolling_perchange'],
            color=color,
            lw=5,
            zorder=3
        )

    maxima.append(max(df_ts_weekly_class_density_plot['rolling_perchange']))
    minima.append(min(df_ts_weekly_class_density_plot['rolling_perchange']))

# Zero reference line
ax.plot(
    np.arange(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    np.zeros(len(df_ts_weekly_class_density_plot['rolling_perchange']) * 7),
    linestyle=':',
    color='k'
)

# Background shading from stringency index
stringencies = []
for date in df_ts_class_density['date']:
    stringencies.append(
        df_stringency[df_stringency['date'] == date]
        .reset_index(drop=True)
        .loc[0, 'stringency_index']
    )

ymin = -105
ymax = 30

for k in range(len(df_ts_class_density)):
    try:
        rgba = matplotlib.cm.gray(
            1 - (stringencies[k] - min(stringencies)) / max(stringencies)
        )
    except:
        rgba = matplotlib.cm.gray(
            1 - (stringencies[k - 1] - min(stringencies)) / max(stringencies)
        )
    x = [k - 0.5, k + 0.5]
    ax.fill_between(x, ymin, ymax, color=rgba, alpha=0.4, edgecolor='None', linewidth=0, zorder=0)

# Custom x-ticks
xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'Apr 2022']
for i in range(0, len(df_ts_class_density['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=23, pad=6, rotation=90)

# Custom y-ticks
yticks = []
for i in range(ymin, ymax):
    if i % 25 == 0:
        yticks.append(i)
ax.set_yticks(yticks, yticks)
for y in yticks:
    ax.plot(
        [0, len(df_ts_class_density['rolling_perchange'])],
        [y, y],
        color='gray',
        lw=0.7,
        zorder=0
    )
ax.tick_params(axis='y', labelsize=23, pad=6, rotation=0)

# Save and show
plt.savefig(
    wd + '/plots/evolution/' + flows + '/by-density/' + country_short +
    '/evo_netflows' + capital_suffix + dist_suffix + raw_suffix +
    adjust_suffix + radius_suffix + '_no_baseline.pdf',
    bbox_inches='tight'
)

plt.show()

# Transforming the trend DataFrame to long format and saving it as CSV

In [ ]:
# Initialise trend DataFrame with weeks as columns
df_trend = pd.DataFrame(
    columns=[df_ts_weekly_inflows.loc[i, 'week_start'] for i in range(len(df_ts_weekly_inflows))]
)

# Compute trend component for each class density
for i in range(n_class_density):
    netflows_class_density = (
        df_ts_weekly_inflows_class_density[i, :] - df_ts_weekly_outflows_class_density[i, :]
    )
    netflows_baseline_class_density = (
        df_ts_weekly_inflows_baseline_class_density[i, :] - df_ts_weekly_outflows_baseline_class_density[i, :]
    )

    netflows_perchange = [
        (netflows_class_density[j] - netflows_baseline_class_density[j]) /
        netflows_baseline_class_density[j] * 100
        for j in range(len(netflows_baseline_class_density))
    ]

    series = pd.DataFrame({
        'index': pd.to_datetime(df_ts_weekly_inflows['week_start']),
        'value': netflows_perchange
    })
    series.set_index('index', inplace=True)

    try:
        result = seasonal_decompose(series, model='additive', extrapolate_trend='freq')
        df_trend.loc[i] = result.trend.values
    except Exception as e:
        print(f'Not possible for class {i}: {e}')

# Convert to long format
time = []
cat = []
y = []

for i in range(len(df_trend)):
    for j, col in enumerate(df_trend.columns):
        time.append(j)
        cat.append(i)
        y.append(df_trend.loc[i, col])

df_trend_long = pd.DataFrame({
    'time': time,
    'cat': cat,
    'y': y
})

# Save to CSV
df_trend_long.to_csv(
    wd + '/data/outputs/' + country_short + '/mov-analysis/by-density/trend' + capital_suffix + '.csv',
    index=False
)

# Transforming the time series to long format and saving it as CSV

In [ ]:
# Create DataFrame to hold time series data (rows: density classes, columns: week_start dates)
df_ts = pd.DataFrame(
    columns=[df_ts_weekly_inflows.loc[i, 'week_start'] for i in range(len(df_ts_weekly_inflows))]
)

# Calculate netflow % changes for each density class and store in df_ts
for i in range(n_class_density):
    netflows_class_density = (
        df_ts_weekly_inflows_class_density[i, :] - df_ts_weekly_outflows_class_density[i, :]
    )
    netflows_baseline_class_density = (
        df_ts_weekly_inflows_baseline_class_density[i, :] - df_ts_weekly_outflows_baseline_class_density[i, :]
    )

    netflows_perchange = [
        (netflows_class_density[j] - netflows_baseline_class_density[j]) /
        netflows_baseline_class_density[j] * 100
        for j in range(len(netflows_baseline_class_density))
    ]

    series = pd.DataFrame({
        'index': pd.to_datetime(df_ts_weekly_inflows['week_start']),
        'value': netflows_perchange
    })
    series.set_index('index', inplace=True)

    try:
        seasonal_decompose(series, model='additive', extrapolate_trend='freq')
        df_ts.loc[i] = series['value']
    except Exception as e:
        print(f'Not possible for class {i}: {e}')

# Reshape from wide to long format
time = []
cat = []
y = []

for i in range(len(df_ts)):
    for j, col in enumerate(df_ts.columns):
        time.append(j)
        cat.append(i)
        y.append(df_ts.loc[i, col])

df_ts_long = pd.DataFrame({
    'time': time,
    'cat': cat,
    'y': y
})

# Save to CSV
df_ts_long.to_csv(
    wd + '/data/outputs/' + country_short + '/mov-analysis/by-density/time_series' + capital_suffix + '.csv',
    index=False
)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(len(netflows_perchange)), netflows_perchange)
ax.plot(np.arange(len(netflows_perchange)), result.trend)
ax.plot(np.arange(len(netflows_perchange)), result.seasonal)
ax.plot(np.arange(len(netflows_perchange)), df_trend.loc[0])
ax.plot(np.arange(len(netflows_perchange)), df_trend.loc[1])
ax.plot(np.arange(len(netflows_perchange)), df_trend.loc[2])
ax.plot(np.arange(len(netflows_perchange)), df_trend.loc[3])

In [ ]:
fig, ax = plt.subplots()

# Axis style
ax.tick_params(axis='both', which='both', width=0, length=0, color='k', labelsize=20, pad=9)

# Colormap setup
viridis = plt.cm.get_cmap('viridis')
norm = plt.Normalize(0, n_class_density - 1)

maxima = []
minima = []

for i in range(n_class_density):
    color = viridis(norm(i)) 
    
    # Compute absolute netflows
    netflows_class_density = df_ts_weekly_inflows_class_density[i, :] - df_ts_weekly_outflows_class_density[i, :]
    netflows_baseline_class_density = df_ts_weekly_inflows_baseline_class_density[i, :] - df_ts_weekly_outflows_baseline_class_density[i, :]

    df_plot = pd.DataFrame({'perchange_class': netflows_class_density})
    df_baseline_plot = pd.DataFrame({'perchange_class': netflows_baseline_class_density})
    
    # Smooth data using EWM and Savitzky-Golay filter
    df_plot['rolling_perchange'] = df_plot['perchange_class'].ewm(span=10).mean()
    df_plot['rolling_perchange'] = scipy.signal.savgol_filter(df_plot['rolling_perchange'], window_length=10, polyorder=3)
    
    df_baseline_plot['rolling_perchange'] = df_baseline_plot['perchange_class'].ewm(span=10).mean()
    df_baseline_plot['rolling_perchange'] = scipy.signal.savgol_filter(df_baseline_plot['rolling_perchange'], window_length=10, polyorder=3)
    
    # Plot density class curve
    ax.plot(np.arange(len(df_plot['rolling_perchange'])) * 7, df_plot['rolling_perchange'], color=color, lw=5, zorder=3)
    # Optional: Baseline dashed line
    # ax.plot(np.arange(len(df_baseline_plot['rolling_perchange'])) * 7, df_baseline_plot['rolling_perchange'], color=color, lw=2, linestyle=':', zorder=3)

    maxima.append(max(df_plot['rolling_perchange']))
    minima.append(min(df_plot['rolling_perchange']))

# Optional zero line
# ax.plot(np.arange(len(df_plot['rolling_perchange']) * 7), np.zeros(len(df_plot['rolling_perchange']) * 7), linestyle=':', color='k')

# Background shading: stringency index
stringencies = []
for date in df_ts_class_density['date']:
    try:
        stringency = df_stringency[df_stringency['date'] == date].reset_index(drop=True).loc[0, 'stringency_index']
    except:
        stringency = df_stringency[df_stringency['date'] == date].reset_index(drop=True).loc[0, 'stringency_index']
    stringencies.append(stringency)

ymin = int(min(minima)) - 2500
ymax = int(max(maxima)) + 2500

for k in range(len(df_ts_class_density)):
    try:
        rgba = matplotlib.cm.gist_heat(1 - (stringencies[k] - min(stringencies)) / max(stringencies))
    except:
        rgba = matplotlib.cm.gist_heat(1 - (stringencies[k - 1] - min(stringencies)) / max(stringencies))
    
    ax.fill_between([k - 0.5, k + 0.5], ymin, ymax, color=rgba, alpha=0.6, edgecolor='None', linewidth=0, zorder=0)

# X-axis ticks: every ~6 months
xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'Apr 2022']
for i in range(0, len(df_ts_class_density['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=23, pad=6, rotation=90)

# Y-axis ticks
yticks = [i for i in range(ymin, ymax) if i % 10000 == 0]
ax.set_yticks(yticks, yticks)
for y in yticks:
    ax.plot([0, len(df_ts_class_density['rolling_perchange'])], [y, y], color='gray', lw=0.7, zorder=0)
ax.tick_params(axis='y', labelsize=23, pad=6)

# plt.savefig(wd + '/plots/evolution/' + flows + '/by-density/' + country_short + '/evo_netflows_absolute' + capital_suffix + dist_suffix + raw_suffix + adjust_suffix + radius_suffix + '.pdf', bbox_inches='tight')

plt.show()